# Warlords Multi-Agent Toernooi
Dit notebook draait de Warlords-omgeving vanuit de Arcade Learning Environment (ALE) met 4 custom agenten.

Dit notebook is ontworpen om te draaien in Google Colab. Als je hem lokaal draait, moet je mogelijk meer libraries installeren en controleren of hun versies compatibel zijn.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1 Installeer libraries

Voer eerst de onderstaande codecell uit om de libraries te installeren.

In [1]:
# Install the necessary libraries
# !set CMAKE_TOOLCHAIN_FILE=C:\Users\Mark\vcpkg\scripts\buildsystems\vcpkg.cmake
!pip install pettingzoo[atari]  --no-cache-dir
!pip install "autorom[accept-rom-license]"
!pip install --find-links dist/ --no-cache-dir AutoROM[accept-rom-license]

Using legacy 'setup.py install' for multi-agent-ale-py, since package 'wheel' is not installed.
    Running setup.py install for multi-agent-ale-py: started
    Running setup.py install for multi-agent-ale-py: finished with status 'error'


    ERROR: Command errored out with exit status 1:
     command: 'c:\users\mark\documents\school\ads\autonomous_systems\autonomous_3\.venv\scripts\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Mark\\AppData\\Local\\Temp\\pip-install-0xy5ekpy\\multi-agent-ale-py\\setup.py'"'"'; __file__='"'"'C:\\Users\\Mark\\AppData\\Local\\Temp\\pip-install-0xy5ekpy\\multi-agent-ale-py\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\Mark\AppData\Local\Temp\pip-record-yeue0uku\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\users\mark\documents\school\ads\autonomous_systems\autonomous_3\.venv\include\site\python3.9\multi-agent-ale-py'
         cwd: C:\Users\Mark\AppData\Local\Temp\pip-install-0xy5ekpy\multi-agent-ale-py\
    Complete output (74 lines):
    running install
   

You should consider upgrading via the 'c:\users\mark\documents\school\ads\autonomous_systems\autonomous_3\.venv\scripts\python.exe -m pip install --upgrade pip' command.


Looking in links: dist/


You should consider upgrading via the 'c:\users\mark\documents\school\ads\autonomous_systems\autonomous_3\.venv\scripts\python.exe -m pip install --upgrade pip' command.


**Herstart nu je kernel**. Na het herstarten kun je direct doorgaan met de volgende codecel.

## 2 Importeer libraries en download Atari ROMs

Je krijgt een prompt om de AutoROM-overeenkomst te accepteren. Druk op "Y" wanneer je dit ziet.

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126 
!pip install opencv-python-headless --no-cache-dir

Looking in indexes: https://download.pytorch.org/whl/cu126
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     -------------------------------------- 536.2/536.2 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 30.7 MB/s eta 0:01:29
   ---------------------------------------- 0.0/2.7 GB 37.5 MB/s eta 0:01:12
   ---------------------------------------- 0.0/2.7 GB 41.0 MB/s eta 0:01:06
    --------------------------------------- 0.0/2.7 GB 43.4 MB/s eta 0:01:02
    --------------------------------------- 0.0/2.7 GB 44.6 MB/s eta 0:01:00
    --------------------------------------- 0.1/2.7 GB 45.6 MB/s eta 0:00:59
    --------------------------------------- 0.1/2.7 GB 46.0 MB/s eta 0:00:58
   - -------------------------------------- 0.1/2.7 GB 46.5 MB/s eta 0:00:57
   - -------------------------------------- 0.1/2.7 GB 46.7 MB/s eta 0:00:57
   - ---------------------

In [ ]:
# Start AutoROM

!AutoROM -y

# Import libraries
from pettingzoo.atari import warlords_v3
from pettingzoo.utils import BaseParallelWrapper
import gymnasium as gym
import numpy as np
from collections import defaultdict, Counter
import importlib
import os
import imageio

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.11/dist-packages/AutoROM/roms
	/usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms/amidar.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/assault.bin
Installed /usr/local/lib/python3.11/dist-packages/multi_

## 3 Initialiseer agenten

In deze codecel importeren we de AI-agenten om Warlords te spelen. De bestanden met de agentklassen moeten zich in dezelfde map bevinden als dit notebook.

In [ ]:
# Import the agent classes
# The BaselineAgent is defined in the first cell of this notebook.
from src.agents.baseline_agent import BaselineAgent # This might be redundant or point to the same

# Import our new DQNAgent
from src.agents.dqn_agent import DQNAgent
print("DQNAgent imported successfully.")


from collections import defaultdict, Counter # Already imported, but good practice if cell is standalone

agent_names = ['Agent1', 'Agent2', 'Agent3', 'Agent4']

# Get observation and action space dimensions for DQNAgent
# We need to create a temporary environment for this
try:
    temp_env_for_specs = warlords_v3.env()
    temp_env_for_specs.reset()
    agent_id_example = temp_env_for_specs.agents[0]
    obs_space_shape_raw = temp_env_for_specs.observation_space(agent_id_example).shape # Raw shape (e.g., 210, 160, 3)
    action_space_n = temp_env_for_specs.action_space(agent_id_example).n
    temp_env_for_specs.close()
    print(f"Observation space shape: {obs_space_shape_raw}, Action space size: {action_space_n}")

    # Hyperparameters for DQNAgent
    # Reduced sizes for quick demonstration, increase for real training
    dqn_hyperparams = {
        "buffer_size": 10000,          # Size of replay buffer
        "batch_size": 32,              # Batch size for training
        "gamma": 0.99,                 # Discount factor
        "eps_start": 1.0,              # Initial epsilon for exploration
        "eps_end": 0.1,                # Final epsilon
        "eps_decay_steps": 30000,      # How many agent steps to decay epsilon
        "learning_rate": 0.00025,       # Learning rate for Adam optimizer
        "target_update_frequency": 500,# How often to update target network (agent steps)
        "learn_start_steps": 1000,     # Agent steps before learning starts
        "frame_stack_size": 4,         # Number of frames to stack
        "img_height": 84,              # Preprocessed image height
        "img_width": 84,               # Preprocessed image width
    }

    # Instantiate each agent
    agent_instances = [
        DQNAgent(player_name=agent_names[0],
                 input_shape_raw=obs_space_shape_raw,
                 num_actions=action_space_n,
                 **dqn_hyperparams),
        BaselineAgent(agent_names[1]), # Random agent
        BaselineAgent(agent_names[2]), # Random agent
        BaselineAgent(agent_names[3])  # Random agent
    ]
    print(f"Instantiated {agent_instances[0].name} as DQNAgent.")
    for i in range(1,4):
        print(f"Instantiated {agent_instances[i].name} as BaselineAgent.")

except Exception as e:
    print(f"Could not initialize agents with DQNAgent, falling back to all BaselineAgent: {e}")
    agent_instances = [
        BaselineAgent(agent_names[0]),
        BaselineAgent(agent_names[1]),
        BaselineAgent(agent_names[2]),
        BaselineAgent(agent_names[3])
    ]


scores = defaultdict(int) # This will store scores for the *current* game/episode
wins = Counter() # This will store "points" won, accumulated across games

DQNAgent imported successfully.
Observation space shape: (210, 160, 3), Action space size: 6
DQNAgent 'Agent1' using device: cpu
Instantiated Agent1 as DQNAgent.
Instantiated Agent2 as BaselineAgent.
Instantiated Agent3 as BaselineAgent.
Instantiated Agent4 as BaselineAgent.


## 4 Speel het spel

In deze sectie spelen de vier agenten Warlords. Aan het einde van elk spel wordt de score bijgehouden. De winnaar is de agent met de hoogste score.

In [ ]:
# Create environment
env = warlords_v3.env(render_mode="rgb_array")

# Prepare directory for videos
video_dir = os.path.join("warlords_videos")
os.makedirs(video_dir, exist_ok=True)


De volgende codecel speelt het spel.

In [ ]:
# Function to run one game and record video
def run_game(game_id, training_mode=True, video_path_prefix="game"): # Added training_mode
    env.reset()

    # Reset episode-specific states for agents (like frame stacks for DQN)
    for agent_instance in agent_instances:
        if hasattr(agent_instance, 'reset_episode_state'):
            agent_instance.reset_episode_state()

    # Map environment agent IDs (e.g., 'first_0') to our agent instances
    agent_mapping = {
        env.agents[i]: agent_instances[i]
        for i in range(len(env.agents))
    }

    # Map environment agent IDs to our custom agent names (e.g., 'Agent1')
    agent_name_mapping = {
        env.agents[i]: agent_names[i]
        for i in range(len(env.agents))
    }

    # Reset scores for this game (as per original notebook logic)
    current_game_scores = defaultdict(int) # Use a local var for current game scores

    # For DQN: store previous state and action to form a full transition (s, a, r, s')
    # Keyed by environment agent ID (e.g., 'first_0')
    agent_prev_s_a = {env_agent_id: {'state': None, 'action': None} for env_agent_id in env.agents}

    frames = [] # For video recording

    for env_agent_id in env.agent_iter(): # env_agent_id is like 'first_0', 'second_0'
        observation_raw, reward, termination, truncation, info = env.last()

        agent_obj = agent_mapping[env_agent_id]
        agent_custom_name = agent_name_mapping[env_agent_id] # 'Agent1', 'Agent2', ...

        current_game_scores[agent_custom_name] += reward # Accumulate reward for this game/episode

        if reward > 0: # Original "win" condition (scoring a point)
            print(f"Point for {agent_custom_name}! (Total points for {agent_custom_name} this game: {current_game_scores[agent_custom_name]})")
            wins[agent_custom_name] += 1 # Global "points" counter

        done = termination or truncation

        action = None
        if isinstance(agent_obj, DQNAgent):
            # 1. Process current raw observation into a stacked state (s')
            #    The get_stacked_frames method handles internal frame deque and stacking
            current_stacked_state = agent_obj.get_stacked_frames(observation_raw) # This is s'

            # 2. If there was a previous state (s) and action (a) for this agent,
            #    we can now form a transition (s, a, r, s', done) and store it.
            prev_s_dict_entry = agent_prev_s_a[env_agent_id]
            if prev_s_dict_entry['state'] is not None and prev_s_dict_entry['action'] is not None:
                prev_s = prev_s_dict_entry['state']
                prev_a = prev_s_dict_entry['action']
                agent_obj.replay_buffer.add(prev_s, prev_a, reward, current_stacked_state, done)

                if training_mode:
                    agent_obj.train_step() # Trigger learning

            # 3. If agent is done for this episode:
            if done:
                action = None # PettingZoo expects action, None is okay for terminated agent
                # Clear previous state/action for this agent as episode ended for it
                agent_prev_s_a[env_agent_id]['state'] = None
                agent_prev_s_a[env_agent_id]['action'] = None
            else:
                # 4. Get action for the current_stacked_state (s')
                action = agent_obj.act(current_stacked_state, training=training_mode)
                # 5. Store current_stacked_state (s') and action (a') to be used as s and a in the next turn for this agent
                agent_prev_s_a[env_agent_id]['state'] = current_stacked_state
                agent_prev_s_a[env_agent_id]['action'] = action

        else: # For BaselineAgent or other non-learning agents
            if done:
                action = None
            else:
                action = agent_obj.act(observation_raw) # Baseline acts on raw observation

        env.step(action)

        # Capture the rendered frame for video
        frame = env.render()
        frames.append(frame)

    env.close() # Close environment at the end of the episode

    # Save video using imageio
    video_path = os.path.join(video_dir, f"{video_path_prefix}_{game_id}.mp4")
    try:
        imageio.mimsave(video_path, frames, fps=30)
        print(f"Video saved to {video_path}")
    except Exception as e:
        print(f"Error saving video: {e}")
        print("Frames list length:", len(frames))
        if len(frames) > 0:
            print("Shape of first frame:", frames[0].shape if frames[0] is not None else "None")


    # Return current game scores, to be aggregated by the caller
    # The global `wins` counter is already updated based on immediate positive rewards.
    # The global `scores` in the original notebook seems to store the last game's scores.
    # We will update the global `scores` with this game's scores.
    for agent_n in agent_names: # Update global scores dict with this game's results
        scores[agent_n] = current_game_scores[agent_n]

    # For debugging DQNAgent's epsilon
    for agent_instance in agent_instances:
        if isinstance(agent_instance, DQNAgent):
            print(f"End of Game {game_id}: {agent_instance.name} epsilon = {agent_instance.current_epsilon:.4f}, total steps = {agent_instance.total_steps_taken}")

In [ ]:
# Run N games (episodes)
NUM_GAMES_TO_RUN = 5 # Set to a higher number for more training, e.g., 100, 1000
TRAINING_MODE_ENABLED = True # Set to False to evaluate agent without exploration/learning updates

for game_idx in range(NUM_GAMES_TO_RUN):
    print(f"\nRunning game {game_idx + 1}/{NUM_GAMES_TO_RUN}...")
    # The run_game function now updates the global `scores` and `wins`
    run_game(game_id=game_idx, training_mode=TRAINING_MODE_ENABLED)

print("\n--- Final Results After All Games ---")
# Print accumulated "points" (wins) and scores from the last game
for agent_n in agent_names:
    # `scores[agent_n]` here will reflect the scores from the *last* game played
    # `wins[agent_n]` is the accumulated "points" over all games
    print(f"{agent_n}: Score in last game = {scores[agent_n]}, Total 'points' won = {wins[agent_n]}")

try:
    # The winner definition from the original notebook (agent with most "points")
    winner = wins.most_common(1)[0]
    print(f"Overall Winner (most points): {winner[0]} with {winner[1]} points!")
except IndexError:
    print("No 'points' were scored by any agent.")

# Optionally, save the trained DQN model
for agent_instance in agent_instances:
    if isinstance(agent_instance, DQNAgent):
        model_save_path = os.path.join(DIR, f"{agent_instance.name}_dqn_model.pth")
        agent_instance.save_model(model_save_path)

In [ ]:
# Display download links for videos
# import glob
# from IPython.display import FileLink, display
# video_files = sorted(glob.glob(f"{video_dir}/*.mp4"))
# print("\nDownload the recorded games:")
# for file in video_files:
#     display(FileLink(file))